<a href="https://colab.research.google.com/github/kimiabaheri-cpu/AER850_Project_3/blob/main/Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Name: Kimia Baheri
# Section: 01
# Course: AER 850
# Project No.03

#-------------------------------------------------------------------------------
# Step 1: Object Masking
#-------------------------------------------------------------------------------
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files

# Mount Google Drive
drive.mount('/content/drive')

# Address the motherboard image
image_path = "/content/drive/MyDrive/Notebooks/motherboard_image.JPEG"

# Load the image
original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)

# Rotate the image 90 degrees clockwise for proper orientation
rotated_image = cv2.rotate(original_image, cv2.ROTATE_90_CLOCKWISE)

# Apply blur to reduce noise
blurred_image = cv2.blur(rotated_image, (15, 15))

# Convert to grayscale
grayscale_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)

# Apply adaptive thresholding
thresholded_image = cv2.adaptiveThreshold(
    grayscale_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 55, 10
)

# Detect edges using Canny edge detector
edges_image = cv2.Canny(thresholded_image, 10, 80)

# Dilate the edges to make them thicker and more visible for display
kernel = np.ones((3, 3), np.uint8)
edges_display = cv2.dilate(edges_image, kernel, iterations=3)

# For contour detection, use more dilation
dilated_edges = cv2.dilate(edges_image, kernel, iterations=12)

# Find contours
contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank mask
mask = np.zeros_like(rotated_image)

# Find the largest contour based on area
largest_contour = max(contours, key=cv2.contourArea, default=None)

# Draw the largest contour onto the mask
cv2.drawContours(mask, [largest_contour], -1, (255, 255, 255), thickness=cv2.FILLED)

# Apply the mask to extract the PCB
masked_image = cv2.bitwise_and(rotated_image, mask)

# Display results in subplots
plt.figure(figsize=(15, 5))

# Edge detection image
plt.subplot(1, 3, 1)
plt.imshow(edges_image, cmap="gray")
plt.title("Canny Edge Detection")
plt.axis('off')

# Mask
plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(mask, cv2.COLOR_BGR2RGB))
plt.title("PCB Mask")
plt.axis('off')

# Final masked PCB image
plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))
plt.title("Extracted Motherboard")
plt.axis('off')

plt.tight_layout()
plt.show()

#-------------------------------------------------------------------------------
# Step 2:  YOLOv11 Training
#-------------------------------------------------------------------------------
# Install the ultralytics
!pip install ultralytics

# Import the necessary libraries
from ultralytics import YOLO
from PIL import Image

# Load the YOLOv11 nano pretrained model
model = YOLO('yolo11n.pt')

# Train the model with the PCB dataset
results = model.train(
    data='/content/drive/MyDrive/Notebooks/data/data.yaml',      # Path to the desired dataset
    epochs=180,                     # Number of training epochs (should be below 200)
    batch=8,                        # Batch size
    imgsz=1200,                     # Image size
    name='Step2_Yolov11_Training',   # Name of the training run
    plots = True,                    # Plot training results
    project = '/content/drive/MyDrive/Notebooks/Step2-Results',  # Where to save the trained model
)
print("Training completed!")
print(f"Results saved in: /content/drive/MyDrive/Notebooks/Step2-Results/Step2_Yolov11_Training")

#-------------------------------------------------------------------------------
# Step 3: Evaluation
#-------------------------------------------------------------------------------
# Load the trained model
trained_model = YOLO('/content/drive/MyDrive/Notebooks/Step2-Results/Step2_Yolov11_Training/weights/best.pt')

# Define the paths for evaluation images
arduinomega_path = '/content/drive/MyDrive/Notebooks/data/evaluation/ardmega.jpg'
arduinoUno_path = '/content/drive/MyDrive/Notebooks/data/evaluation/arduno.jpg'
RaspberryPi_path = '/content/drive/MyDrive/Notebooks/data/evaluation/rasppi.jpg'

# Prediction for Arduino Mega
results_mega = trained_model.predict(
    source=arduinomega_path,
    imgsz=2048,
    conf=0.35,
    save=True,
    project='/content/drive/MyDrive/Notebooks/Step3-Results',
    name='arduino_mega',
)

# Prediction for Arduino Uno
results_uno = trained_model.predict(
    source=arduinoUno_path,
    imgsz=2048,
    conf=0.35,
    save=True,
    project='/content/drive/MyDrive/Notebooks/Step3-Results',
    name='arduino_uno',
)

# Prediction for Raspberry Pi
results_raspi = trained_model.predict(
    source=RaspberryPi_path,
    imgsz=2048,
    conf=0.35,
    save=True,
    project='/content/drive/MyDrive/Notebooks/Step3-Results',
    name='raspberry_pi',
)

print("\nEvaluation completed!")
print("Results saved in: /content/drive/MyDrive/Notebooks/Step3-Results")
print(f"\nArduino Mega detections: {len(results_mega[0].boxes)}")
print(f"Arduino Uno detections: {len(results_uno[0].boxes)}")
print(f"Raspberry Pi detections: {len(results_raspi[0].boxes)}")


